In [1]:
import pandas as pd
import numpy as np
import re
import time
import random
import gensim
import jieba
from numpy import *
import json
from numpy import linalg as la  

E:\Applications\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [87]:
wu=pd.read_csv('wvusermatrix.csv',encoding='gbk')#word2vec用户矩阵
wt=pd.read_csv('wvtvmatrix2.csv',encoding='gbk')#word2vec节目矩阵
tu=pd.read_csv('tagusermatrix.csv',encoding='gbk')#标签用户矩阵
cu=pd.read_csv('channelusermatrix.csv',encoding='gbk')#标签用户频道矩阵
tt=pd.read_csv('tagtvmatrix2.csv',encoding='gbk')#标签节目矩阵
dfr=pd.read_csv('mingzilianhe.csv',encoding='gbk')#用户看过的节目
info=pd.read_csv('info3.csv',encoding='gbk').fillna('')#节目信息


for i in range(len(tu)):
    if np.sum(tu.loc[i,:])!=0:
        tu.loc[i,:]=tu.loc[i,:]/np.sum(tu.loc[i,:])
for i in range(len(cu)):
    if np.sum(cu.loc[i,:])!=0:
        cu.loc[i,:]=cu.loc[i,:]/np.sum(cu.loc[i,:])
tu=tu+cu*0.2
tu.loc[:,'新闻']=0.0
        
typelist=['犯罪', '喜剧', '舞台艺术', '新闻', '儿童', '真人秀', '歌舞', '戏曲', '音乐', '冒险', '灾难', '恐怖', '家庭', '短片', '历史', '运动', '动画', '纪录片', '惊悚', '科幻', '战争', '动作', '健康', '脱口秀', '悬疑', '剧情', '古装', '鬼怪', '奇幻', '武侠', '同性', '体育', '西部', '传记', '爱情']
typelist=sorted(typelist)

In [90]:
rate={}
for i in range(len(info)):
    if info.loc[i,'rate']=='':
        rate[info.loc[i,'name']]=6.69
        continue
    di=json.loads(info.loc[i,'rate'])
    if di["average"]>0:
        rate[info.loc[i,'name']]=di["average"]
        print(di["average"])
    else:
        rate[info.loc[i,'name']]=6.69

5.1
9.0
5.2
7.6
6.1
8.3
4.0
9.2
7.6
6.1
4.7
8.6
7.3
5.0
5.7
5.7
8.8
8.3
5.5
9.0
6.3
8.8
8.7
7.3
9.6
5.6
5.6
5.6
5.1
9.2
9.0
8.2
8.3
6.5
7.8
6.5
4.6
2.5
6.6
7.0
7.3
6.2
6.9
6.2
8.2
7.1
7.6
8.0
6.3
8.3
7.8
7.7
9.6
3.6
7.1
7.1
6.6
6.1
6.5
6.2
3.0
6.8
5.4
8.1
4.8
4.7
7.9
9.3
8.3
6.9
7.2
6.5
4.9
8.5
5.4
4.5
6.2
9.2
7.3
3.0
5.4
5.3
6.7
4.7
4.7
8.0
6.4
8.0
8.3
9.7
8.5
8.0
8.8
6.0
7.0
3.3
6.0
7.6
6.2
6.3
4.9
6.4
6.0
4.7
3.2
3.0
6.2
7.9
6.7
6.3
5.4
3.4
8.8
7.8
3.9
8.3
9.4
5.3
7.6
7.8
8.2
6.8
9.5
3.2
6.9
7.4
3.2
7.2
8.7
4.8
7.4
4.7
5.8
7.1
5.9
6.8
4.1
4.1
6.8
5.2
8.2
7.4
7.0
8.3
6.5
5.1
8.2
4.7
6.9
6.5
7.2
7.2
9.4
6.2
7.9
4.6
3.4
4.6
7.2
9.5
4.0
6.9
4.6
7.7
8.4
5.4
7.3
5.8
6.1
5.1
7.4
7.0
8.7
2.6
9.1
7.1
4.4
5.2
7.0
8.0
5.5
8.9
7.3
7.9
3.4
9.1
6.7
4.0
4.9
6.3
8.7
7.9
5.4
5.1
6.2
6.4
8.0
6.3
3.6
6.2
6.9
6.8
3.6
8.4
8.2
8.8
7.3
5.5
5.8
7.2
8.1
4.8
8.0
7.3
4.8
7.2
5.5
8.5
3.0
6.0
7.5
7.7
3.8
6.5
6.7
7.5
6.0
7.8
8.2
7.6
9.1
9.0
8.8
6.1
5.8
5.5
8.1
4.3
6.2
3.1
6.8
4.4
6.1
8.5
5.5
4.6
8.2
6.3
3.7
9.3


In [102]:
#求余弦
def cosSim(inA,inB):
    num = float(np.sum(inA*inB))
    denom = la.norm(inA)*la.norm(inB)
    if denom==0.0:
        return 0.0
    return num/denom

def getRec(u,wu=wu,wt=wt,tu=tu,tt=tt,dfr=dfr,info=info):
    reslist=[]
    ratelist=[]
    #print(u)
    #print(dfr.loc[u,'name'])
    ctlist=[]#记录出现过的标签
    if np.sum(tu.loc[u,:])==0:
        print('该用户无历史数据')
        return [],[]
    ua1=np.array(wu.loc[u,:])
    ua2=np.array(tu.loc[u,:])
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:12]:
        ctlist+=json.loads(info.loc[t[0],'info'])
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])
    
    #第二轮推荐
    ctlist=list(set(ctlist))
    now=pd.DataFrame(np.ones((1,35)),columns=typelist)
    for c in ctlist:
        now.loc[0,c]=0.0
    ua2=np.array(tu.loc[u,:])*np.array(now.loc[0,:])[:35]
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=(cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']])*0.6
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:8]:
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])
    return reslist,ratelist

In [ ]:
#求余弦
def cosSim(inA,inB):
    num = float(np.sum(inA*inB,axis=1))
    denom = la.norm(inA)*la.norm(inB)
    if denom==0.0:
        return 0.0
    return num/denom

def getRec(u,wu=wu,wt=wt,tu=tu,tt=tt,dfr=dfr,info=info):
    reslist=[]
    ratelist=[]
    #print(u)
    #print(dfr.loc[u,'name'])
    ctlist=[]#记录出现过的标签
    if np.sum(tu.loc[u,:])==0:
        print('该用户无历史数据')
        return [],[]
    ua1=np.array(wu.loc[u,:])
    ua2=np.array(tu.loc[u,:])
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']]
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:12]:
        ctlist+=json.loads(info.loc[t[0],'info'])
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])
    
    #第二轮推荐
    ctlist=list(set(ctlist))
    now=pd.DataFrame(np.ones((1,35)),columns=typelist)
    for c in ctlist:
        now.loc[0,c]=0.0
    ua2=np.array(tu.loc[u,:])*np.array(now.loc[0,:])[:35]
    tv=[]
    for i in range(len(wt)):
        tv.append([i,cosSim(ua1,np.array(wt.loc[i,:]))*0.2])
    for i in range(len(tt)):
        tv[i][1]+=(cosSim(ua2,np.array(tt.loc[i,:]))*0.7+0.01*rate[info.loc[i,'name']])*0.6
    tv=sorted(tv,key=lambda k:k[1],reverse=True)
    for t in tv[:8]:
        #print(info.loc[t[0],'name'],info.loc[t[0],'info'],t[1])
        reslist.append(info.loc[t[0],'name'])
        ratelist.append(t[1])
    return reslist,ratelist

In [ ]:
la.norm()

In [13]:
test=pd.read_csv('test/guankan2.csv',encoding='gbk').fillna('')

In [108]:
test

,id,name
0,10001,;味道中国;味道中国;味道中国;味道中国;霍比特人;霍比特人;蜡笔小新;穿越时空的少女;蜡笔...
1,10002,
2,10003,
3,10004,
4,10005,
5,10006,
6,10007,
7,10008,
8,10009,
9,10010,;王牌对王牌;人间至味是清欢;人间至味是清欢;爱情保卫战;爱情保卫战;十二道锋味;十二道锋味...


In [111]:
jg=pd.DataFrame(columns=['用户','产品名称','推荐指数'])
precision=0
cp=0
recall=0
cr=0
n=0
for i in range(len(test)):  
    print(i)
    tuilist,ratelist=getRec(i)
    if len(tuilist)==0:
        continue
    countp=0
    countr=0
    for j in range(20):
        jg.loc[n,'用户']=i+10001
        jg.loc[n,'产品名称']=tuilist[j]
        jg.loc[n,'推荐指数']=ratelist[j]
        n+=1
print('总准确率 召回率',precision/cp,recall/cr)

0


KeyboardInterrupt: 

In [104]:
jg=pd.DataFrame(columns=['用户','产品名称','推荐指数'])
precision=0
cp=0
recall=0
cr=0
n=12280
for i in range(len(test)):  
    print(i)
    kanlist=test.loc[i,'name'].split(';')[1:]
    kanlist=list(set(kanlist))
    tuilist,ratelist=getRec(i)
    if len(tuilist)==0:
        continue
    print(kanlist)
    print(tuilist)
    countp=0
    countr=0
    for t in tuilist:
        if t in kanlist:
            countp+=1
            print(t)
    #precision+=countp/len(tuilist)
    cp+=1
    #print('准确率:',countp/len(tuilist))
    for t in kanlist:
        if t in tuilist:
            countr+=1
    #recall+=countr/len(kanlist)
    cr+=1
    #print('召回率:',countr/len(kanlist))
    for j in range(20):
        jg.loc[n,'用户']=i+10001
        jg.loc[n,'产品名称']=tuilist[j]
        jg.loc[n,'推荐指数']=ratelist[j]
        n+=1
print('总准确率 召回率',precision/cp,recall/cr)

0
['穿越时空的少女', '勇者传说', '与君相恋100次', '蜡笔小新', '摔跤吧！爸爸', '味道中国', '霍比特人', '青衣小怪物']
['小小羊提米提米的怪物', '小老虎丹尼尔 下', '小小羊提米提米的宠物问题', '小小羊提米提米做音乐', '哆啦A梦', '小小羊提米提米的失误', '小小羊提米提米的雪球', '麦兜我和我妈妈', '小小羊提米提米制造波浪', '小小羊提米嘟嘟提米', '小小羊提米提米做买卖', '狐狸与猎狗2', '他是龙', '加勒比海盗5：死无对证', '星际小蚂蚁3D儿歌系列星仔走天涯', 'X战警1', '雷神2：黑暗世界', '蜘蛛侠', '地心历险记2', '魔幻传奇II']
1
[]
['超级飞侠 第二部', '星际小蚂蚁3D儿歌系列DO RE MI', '开心宝贝之开心星星球', '开心宝贝之开心大冒险', '阿U·兔智来了', '星际小蚂蚁3D儿歌系列我们要做雷锋式的好青年', '如意兔之开心农场俺只是个传说', '赛尔号大电影5之雷神崛起', '如意兔之开心农场长翅膀的树', '星际小蚂蚁3D儿歌系列我爱北京天安门', '如意兔之开心农场飞天大王的战车梦', '星际小蚂蚁3D儿歌系列我的好妈妈', '你好，是鹿晗吗 2016', '极限挑战 期', '极限挑战 番外篇期', '中华好故事', '维多利亚的秘密时装内衣秀2016', '我们相爱吧之爱有天意', '160811-02才艺大赛决赛现场视频-高清', '穿越吧厨房2017']
2
[]
['暖男的爱情与战争', '天使的城', '记住我', '生逢灿烂的日子', '今夜天使降临', '大宅门', '太太万岁', '虎妈猫爸', '甜蜜18岁', '蓝色情人节', '消防员', '不是不想嫁', '我们的骄傲', '优？秀', '健康栋笃sell', '跨界喜剧王', '档案：蒋家王朝逃台前最后的日子', '变身超人(陈柏霖变身下岗?潘砍？？', '世界青年说', '今晚80后脱口秀']
3
[]
['你好，是鹿晗吗 2016', '中华好故事', '极限挑战 期', '维多利亚的秘密时装内衣秀2016', '极限挑战 番外篇期', '我想见到你', '我是演说家', '我们相爱吧之爱有天意', '炜哥的味道', '来吧兄弟', '穿越吧厨房

KeyboardInterrupt: 

In [110]:
jg

,用户,产品名称,推荐指数
12280,10717,超级飞侠 第二部,0.857499
12281,10717,开心宝贝之开心星星球,0.845801
12282,10717,星际小蚂蚁3D儿歌系列DO RE MI,0.844769
12283,10717,星际小蚂蚁3D儿歌系列我们要做雷锋式的好青年,0.839675
12284,10717,赛尔号大电影5之雷神崛起,0.838655
12285,10717,如意兔之开心农场俺只是个传说,0.838586
12286,10717,开心宝贝之开心大冒险,0.838306
12287,10717,星际小蚂蚁3D儿歌系列我爱北京天安门,0.836608
12288,10717,如意兔之开心农场飞天大王的战车梦,0.83606
12289,10717,贝瓦儿歌精选集我爱北京天安门,0.835633


In [66]:
allpro=[]
for i in range(1329):
    print(i)
    allpro+=getRec(i)[0]

0
1
2
3
4
5
该用户无历史数据
6
该用户无历史数据
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
该用户无历史数据
25
26
27
28
29
30
31
32
33
34
35
36
37
38
该用户无历史数据
39
40
41
42
43
该用户无历史数据
44
45
该用户无历史数据
46
47
48
49
50
51
52
该用户无历史数据
53
54
55
该用户无历史数据
56
57
58
59
60
61
62
63
64
65
66
67
该用户无历史数据
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
该用户无历史数据
99
100
该用户无历史数据
101
102
103
104
105
106
107
108
109
该用户无历史数据
110
111
112
113
该用户无历史数据
114
115
116
117
118
119
120
121
122
123
124
125
126
该用户无历史数据
127
128
129
该用户无历史数据
130
该用户无历史数据
131
132
该用户无历史数据
133
134
135
136
137
138
139
140
141
该用户无历史数据
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
该用户无历史数据
163
164
该用户无历史数据
165
166
167
168
169
170
171
172
173
该用户无历史数据
174
175
176
177
178
179
180
181
182
183
184
该用户无历史数据
185
186
187
188
189
该用户无历史数据
190
191
192
193
该用户无历史数据
194
195
196
197
198
199
200
201
202
203
204
该用户无历史数据
205
206
207
208
209
210
211
212
213
该用户无历史数据
214
215
216
该用户无历史数据

1303
该用户无历史数据
1304
1305
1306
1307
1308
1309
1310
1311
该用户无历史数据
1312
该用户无历史数据
1313
1314
该用户无历史数据
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328


In [81]:
len(set(allpro))/(2661-18)

0.4911085887249338

In [79]:
from collections import Counter
wrongc=0
countall=0
countcor=0
dictv={}
for i in range(len(info)):
    dictv[info.loc[i,'name']]=json.loads(info.loc[i,'info'])
for i in range(len(test)):
    if test.loc[i,'name']!='':
        print(i)
        kanlist=test.loc[i,'name'].split(';')[1:]
        kanlist=list(set(kanlist))
        tagnow=[]
        for kan in kanlist:
            try:
                tagnow+=dictv[re.sub('\d{8}','',kan)]
            except:
                wrongc+=1
                continue
        taglistn=Counter(tagnow).most_common()
        nrl=[taglistn[0][0]]
        for ta in range(1,len(taglistn)):
            if taglistn[ta][1]<taglistn[ta][1]:
                break
            else:
                nrl.append(taglistn[ta][0])
        if tu.loc[i,:].argmax() in nrl:
            countcor+=1
        countall+=1
print(countcor/countall)

0
9
12
14
21
26
27
28
31
32
35
36
39
44
50
51
53
54
57
62
71
72
73
78
79
89
91
97
105
108
111
114
117
118
122
134
135
139
145
146
148
149
150
152
156
157
159
161
171
178
181
183
185
190
197
201
205
207
208
209
223
224
228
232
233
234
235
236
241
249
253
254
262
264
265
266
271
273
275
279
282
283
289
290
297
298
301
306
307
312
313
316
319
320
326
327
328
331
332
333
334
336
338
342
344
348
350
351
352
355
358
370
375
382
392
395
400
402
407
410
411
412
415
420
423
424
437
441
442
451
453
456
459
467
468
469
475
477
479
485
486
488
489
496
499
500
505
507
510
512
513
514
520
521
522
529
531
536
537
538
539
541
544
545
546
547
548
549
551
552
554
556
564
565
571
574
578
580
584
586
596
599
603
604
610
611
623
629
633
639
640
641
651
655
659
661
662
663
664
683
685
686
689
702
711
717
718
724
728
731
734
735
736
745
750
751
756
758
759
769
773
776
778
779
780
781
787
791
795
796
953
956
969
974
982
987
991
999
1000
1001
1011
1019
1031
1034
1036
1039
1041
1044
1046
1052
1057
1061
1065
107

In [72]:
tu.loc[0,:].argmax()

'真人秀'

In [83]:
tu

,传记,体育,健康,儿童,冒险,剧情,动作,动画,历史,古装,...,真人秀,短片,科幻,纪录片,脱口秀,舞台艺术,西部,运动,音乐,鬼怪
0,0.011895,0.000000,0.000000,0.000000,0.055333,0.109251,0.061485,0.203506,0.000000,0.000000,...,0.069357,0.000000,0.063206,0.083349,0.000000,0.000000,0.000000,0.016002,0.000000,0.0
1,0.000000,0.008975,0.000000,0.027905,0.000000,0.013494,0.000807,0.027905,0.021616,0.008393,...,0.028441,0.000000,0.000000,0.006706,0.001021,0.000000,0.000000,0.008235,0.002647,0.0
2,0.000000,0.000000,0.004581,0.000000,0.000000,0.032317,0.009416,0.000000,0.010408,0.001282,...,0.003549,0.000000,0.000000,0.011761,0.015084,0.001866,0.000000,0.000000,0.000000,0.0
3,0.000000,0.012494,0.003110,0.003789,0.000000,0.017328,0.001746,0.002168,0.018676,0.002125,...,1.032017,0.000000,0.000000,0.014003,0.003853,0.001493,0.000000,0.004389,0.004395,0.0
4,0.000000,0.002057,0.001306,0.002851,0.000000,0.013954,0.001033,0.000000,0.026556,0.006179,...,0.044855,0.000000,0.000000,0.010286,0.003090,0.001103,0.000000,0.009775,0.004105,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,0.000000,0.044028,0.000000,0.009796,0.000000,0.030168,0.000000,0.009755,0.004151,0.000885,...,0.062838,0.000000,0.000000,0.000211,0.000000,0.000000,0.000000,0.022967,0.001301,0.0
8,0.000000,0.106139,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.046931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000379,0.004155,0.000921,0.000000,0.041045,0.000000,0.000921,0.016463,0.002305,...,0.279682,0.000000,0.000000,0.007407,0.003092,0.000000,0.000000,0.012051,0.039404,0.0


In [107]:
jg.to_csv('result/201800010问题一新出现推荐结果表.csv',index=False)